# Facial Recognition: Transfer Learning Video Capture

### Libraries Used:

In [1]:
# Libraries used for OpenCv and path creation/reading
import cv2
import os
from os import listdir

# Keras libraries necessary for the model creation
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation
from keras.layers import Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

# Keras library for VGG16 model
from keras.applications.vgg16 import VGG16

# Libraries for preprocessing the data
from PIL import Image
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array, save_img
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.utils import to_categorical
from glob import glob

# Importing numpy to work with arrays, matplotlib for visuals
# and pandas for dataframe.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy import spatial
import math

In [2]:
# This function finds the cosine similarity between the dataset image
# and the captured images
def cosineSimilarity(dataset_img, captured_img):
    cos_dis = spatial.distance.cosine(dataset_img, captured_img)
    cos_sim = 1 - cos_dis
    return cos_sim

In [3]:
# Load the FR TL model
tl_model = load_model('fr_model.h5')

In [4]:
user_names = []
name = r'C:\Users\jpasz\Facial Recognition TVT\train'
# print(os.listdir(name))
# This list comprehension stores all of the user names into a list.
user_names = [i for i in os.listdir(name)]
print(user_names)

['Jacob Paszkiewicz', 'Jake Cross', 'Txomin Chivite']


In [5]:
# Initialize a file name and dictionary
user_image_path = r'C:/Users/jpasz/Facial Recognition Images'
user_images = dict()
# Iterate a folder ("New Faces") to folder ("User name") to user's 150 face images
# String beocomes file path by listdir
for i in listdir(user_image_path):
    if i != '.DS_Store':  # for mac users
        current_folder = user_image_path + '/' + i
        print(current_folder)
        # initialize and reset a list
        image_predictions = []
        # Creates the test data with imagedatagenerator
        temp_i = ImageDataGenerator(rescale=1.0/255.0)
        i_data = temp_i.flow_from_directory(user_image_path, classes=[i], target_size=(224, 224),
                                             batch_size=16, class_mode='categorical')
        i_img, i_labels = next(i_data) # load images inside of imagedatagenerator
        pred = tl_model.predict(i_img) # get values from each images by model.predict
        # print(pred)
        # print(pred[0])
        print(user_names[pred[0].argmax()])
        user_images[i] = pred # assign the list to dict

C:/Users/jpasz/Facial Recognition Images/Jacob Paszkiewicz
Found 150 images belonging to 1 classes.
Jacob Paszkiewicz
C:/Users/jpasz/Facial Recognition Images/Jake Cross
Found 108 images belonging to 1 classes.
Jake Cross
C:/Users/jpasz/Facial Recognition Images/Txomin Chivite
Found 150 images belonging to 1 classes.
Txomin Chivite


In [10]:
# The webcamera is opened up with OpenCV.
camera = cv2.VideoCapture(0)
while(True):
    result, image = camera.read() # get a image from webcam capture
    # If the image capture is sucessful, this if statement is run
    if result:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
        detected_faces = face_cascade.detectMultiScale(image, 1.1, 4) # Find faces in the captured image
        for (x, y, w, h) in detected_faces: # x, y represents initial posions in a graph. w is width and h is height
            user_detected = False
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            face_recognized = image[y:y + h, x:x + w] # crop detected face
            face_recognized = cv2.resize(face_recognized, dsize=(224, 224)) # resize to 224x224
            face_array = img_to_array(face_recognized)
            face_expanded = np.expand_dims(face_array, axis=0)
            face_preprocessed = preprocess_input(face_expanded)
            # Assign the image extracted from the webcam
            video_prediction = tl_model.predict(face_preprocessed)
            print(video_prediction[0])
            # i is a key of dict which is user's name
            for i in user_images:
                user_folder = user_images[i]
                for j in user_folder:
                    cosine_similarity = cosineSimilarity(j, video_prediction[0])
                    # If cosine similarity is greater than 0.95, then consider that detected
                    # face as the user in folder k, and increment the counter.
                    if(cosine_similarity >= 0.95):
                        # print(cosine_similarity)
                        cv2.putText(image, user_names[video_prediction[0].argmax()],
                                    (int(x+w+20), int(y-15)), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 0), 1)
                        user_detected=True
                        break
             # If the detected image is not in user's face dataset, then display unauthorized
            if(user_detected==False):
                cv2.putText(image, 'Unrecogized User', (int(x+w+20), int(y-15)), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 0), 1)
                print('Unrecognized User')
    # Displays the webcamera to the user.
    cv2.imshow('Facial Recognition', image)
    if cv2.waitKey(1) & 0xFF == ord('s'):
        break
# These two lines shut down OpenCV
camera.release()
cv2.destroyAllWindows()           
            
            
        


[0.7477066  0.25229335 0.         0.        ]
Unrecognized User
[1.000000e+00 8.551457e-25 0.000000e+00 0.000000e+00]
[1.0000000e+00 3.5974733e-22 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 2.2309135e-13 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 1.1089385e-11 0.0000000e+00 0.0000000e+00]
[9.9985003e-01 1.4996219e-04 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 1.2795548e-10 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 2.6465724e-10 0.0000000e+00 0.0000000e+00]
[9.9999702e-01 2.9484102e-06 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 2.0774679e-11 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 1.9405663e-21 0.0000000e+00 0.0000000e+00]
[1.000000e+00 8.173524e-12 0.000000e+00 0.000000e+00]
[1.0000000e+00 3.7584196e-13 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 1.4794133e-26 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 3.5346623e-10 0.0000000e+00 0.0000000e+00]
[1.0000000e+00 1.6892413e-17 0.0000000e+00 0.0000000e+00]
[1.000000e+00 4.679924e-22 0.000000e+00 0.000000e+00]
[1.0000000e+00 1.213